In [1]:
from pyspark import SparkContext

sc = SparkContext("local", "pyspark")
links = sc.textFile("links").map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1:])).partitionBy(2).persist()
ranks = sc.textFile("ranks").map(lambda x: x.split("\t")).map(lambda x: (x[0], float(x[1])))

ranks.collect()

[(u'a', 0.2), (u'b', 0.2), (u'c', 0.2), (u'e', 0.2), (u'd', 0.2)]

In [2]:
links.collect()

[(u'a', [u'b', u'd']),
 (u'c', [u'd']),
 (u'e', [u'a', u'b', u'c']),
 (u'b', [u'c', u'e']),
 (u'd', [u'e'])]

All pages have the same rank (0.2)

In [4]:
MAX = 20

for c in range(MAX): 
    contributions = links.join(ranks).flatMap(lambda x: [(i, x[1][1]/len(x[1][0])) for i in x[1][0]])
    ranks = contributions.reduceByKey(lambda x, y: x + y)

ranks.collect()

[(u'd', 0.23678682235503043),
 (u'a', 0.10530158257403283),
 (u'c', 0.18422267372257844),
 (u'e', 0.3157849425189383),
 (u'b', 0.1579039788294203)]

The pages' ranks converge after 20 iterations.